In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

from nba_api.stats.endpoints import teamgamelogs
from nba_api.stats.static import teams
nba_teams = teams.get_teams()
teamIDs = [team['id'] for team in nba_teams]

In [2]:
games = []
games.append(pd.read_excel(r'.\data\nba odds 2014-15.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2015-16.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2016-17.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2017-18.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2018-19.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2019-20.xlsx'))
games.append(pd.read_excel(r'.\data\nba odds 2020-21.xlsx'))

In [3]:
for i in range(len(games)):
    games[i] = games[i].drop(columns=['Rot','1st','2nd','3rd','4th','Final','Close','2H',], axis=1)

In [4]:
games[0].head()

,Date,VH,Team,Open,ML
0,2014-10-28,V,Dallas,206.5,140
1,2014-10-28,H,SanAntonio,6,-165
2,2014-10-28,V,Orlando,196,380
3,2014-10-28,H,NewOrleans,9.5,-475
4,2014-10-28,V,Houston,6.5,-300


In [5]:
homes = []
visitors = []
for i in range(len(games)):
    homes.append(games[i][games[i]['VH'] == 'H'].reset_index(drop=True))
    visitors.append(games[i][games[i]['VH'] == 'V'].reset_index(drop=True))


In [6]:
for i in range(len(homes)):
    homes[i]['Open'] = np.where(homes[i]['Open'] == 'pk', 0, homes[i]['Open'])
    homes[i]['Open'] = pd.to_numeric(homes[i]['Open'])
    homes[i]['OU'] = np.where(homes[i]['Open'] > 100, homes[i]['Open'], None)
    homes[i]['Spread'] = np.where(homes[i]['Open'] < 100, homes[i]['Open'], None)
    
    visitors[i]['Open'] = np.where(visitors[i]['Open'] == 'pk', 0, visitors[i]['Open'])
    visitors[i]['Open'] = np.where(visitors[i]['Open'] == 'PK', 0, visitors[i]['Open'])
    visitors[i]['Open'] = pd.to_numeric(visitors[i]['Open'])
    visitors[i]['OU'] = np.where(visitors[i]['Open'] > 100, visitors[i]['Open'], None)
    visitors[i]['Spread'] = np.where(visitors[i]['Open'] < 100, visitors[i]['Open'], None)
    
    homes[i] = homes[i].drop(columns=['VH','Open'], axis=1).rename(columns={'Team':'Home', 'ML':'MLH'})
    visitors[i] = visitors[i].drop(columns=['VH','Open'], axis=1).rename(columns={'Team':'Away', 'ML':'MLA'})
    
    games[i] = visitors[i].combine_first(homes[i])

In [7]:
teamMap = {'Atlanta':'ATL', 'Brooklyn':'BKN', 'Boston':'BOS', 'Charlotte':'CHA', 'Chicago':'CHI', 'Cleveland':'CLE', 'Dallas':'DAL', 'Denver':'DEN', 'Detroit':'DET',
       'GoldenState':'GSW', 'Houston':'HOU', 'Indiana':'IND', 'LAClippers':'LAC', 'LALakers':'LAL', 'Memphis':'MEM', 'Miami':'MIA', 'Milwaukee':'MIL', 'Minnesota':'MIN',
       'NewOrleans':'NOP', 'NewYork':'NYK', 'OklahomaCity':'OKC', 'Orlando':'ORL', 'Philadelphia':'PHI', 'Phoenix':'PHX', 'Portland':'POR', 'Sacramento':'SAC', 'SanAntonio':'SAS',
       'Toronto':'TOR', 'Utah':'UTA', 'Washington':'WAS'}

for i in range(len(games)):
    games[i] = games[i].replace({'Away':teamMap,'Home':teamMap})

In [8]:
games[0].to_csv(r'.\data\2014-15 bets.csv')
games[1].to_csv(r'.\data\2015-16 bets.csv')
games[2].to_csv(r'.\data\2016-17 bets.csv')
games[3].to_csv(r'.\data\2017-18 bets.csv')
games[4].to_csv(r'.\data\2018-19 bets.csv')
games[5].to_csv(r'.\data\2019-20 bets.csv')
games[6].to_csv(r'.\data\2020-21 bets.csv')

In [9]:
games[0].head(50)

,Away,Date,Home,MLA,MLH,OU,Spread
0,DAL,2014-10-28,SAS,140,-165.0,206.5,6
1,ORL,2014-10-28,NOP,380,-475.0,196,9.5
2,HOU,2014-10-28,LAL,-300,250.0,209,6.5
3,MIL,2014-10-29,CHA,315,-390.0,192,9
4,PHI,2014-10-29,IND,290,-350.0,194,8.5
5,BKN,2014-10-29,BOS,-105,-115.0,191,1.5
6,ATL,2014-10-29,TOR,161,-190.0,199.5,5.5
7,WAS,2014-10-29,MIA,180,-210.0,194,4.5
8,CHI,2014-10-29,NYK,-195,166.0,190,4
9,MIN,2014-10-29,MEM,290,-350.0,198,8.5
